In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    !pip install -q -U tensorflow-addons
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "nlp"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

No GPU was detected. LSTMs and CNNs can be very slow without a GPU.


In [2]:
shake='https://homl.info/shakespeare'
filepath=keras.utils.get_file('shakespeare.txt',shake)
with open (filepath) as f:
    shaketext=f.read()

1122304/1115394 [==============================] - 0s 0us/step


In [5]:
token=keras.preprocessing.text.Tokenizer(char_level=True)
token.fit_on_texts(shaketext)

In [6]:
token.texts_to_sequences(['first'])

[[20, 6, 9, 8, 3]]

In [7]:
token.sequences_to_texts([[20, 6, 9, 8, 3]])

['f i r s t']

In [8]:
maxid=len(token.word_index)
datasetsize=token.document_count

In [9]:
[encoded]=np.array(token.texts_to_sequences([shaketext]))-1

In [33]:
trainsize=datasetsize*90//100
dataset=tf.data.Dataset.from_tensor_slices(encoded[:trainsize])

In [34]:
nstep=100
windowlength=nstep+1
dataset=dataset.repeat().window(windowlength,shift=1,drop_remainder=True)

<WindowDataset shapes: DatasetSpec(TensorSpec(shape=(None,), dtype=tf.int32, name=None), TensorShape([])), types: DatasetSpec(TensorSpec(shape=(None,), dtype=tf.int32, name=None), TensorShape([]))>

In [35]:
dataset=dataset.flat_map(lambda window:window.batch(windowlength))

In [36]:
dataset

<FlatMapDataset shapes: (None,), types: tf.int32>

In [37]:
batchsize=32
dataset=dataset.shuffle(10000).batch(batchsize)
dataset=dataset.map(lambda window:(window[:,:-1],window[:,1:]))

In [38]:
dataset

<MapDataset shapes: ((None, None), (None, None)), types: (tf.int32, tf.int32)>

In [39]:
dataset.map(lambda xbatch,ybatch:(tf.one_hot(xbatch,depth=maxid),ybatch))

<MapDataset shapes: ((None, None, 39), (None, None)), types: (tf.float32, tf.int32)>

In [40]:
dataset=dataset.prefetch(1)

In [41]:
model=keras.models.Sequential([
    keras.layers.GRU(128,return_sequences=True,input_shape=[None,maxid],dropout=0.2,recurrent_dropout=0.2),
    keras.layers.GRU(128,return_sequences=True,dropout=0.2,recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(maxid,activation='softmax'))
])

In [42]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam')

In [43]:
history=model.fit(dataset,epochs=5)

Epoch 1/5


ValueError: in user code:

    C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:805 train_function  *
        return step_function(self, iterator)
    C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:788 run_step  **
       

In [47]:
dataset=tf.data.Dataset.from_tensor_slices(encoded[:trainsize])
dataset=dataset.window(windowlength,shift=nstep,drop_remainder=True)


In [48]:
dataset=dataset.flat_map(lambda window:window.batch(windowlength))
dataset=dataset.batch(1)
dataset=dataset.map(lambda windows:(windows[:,:-1],windows[:,1:]))
dataset=dataset.map(lambda xbatch,ybatch:(tf.one_hot(xbatch,depth=maxid),ybatch))
dataset=dataset.prefetch(1)

In [50]:
model=keras.models.Sequential([
    keras.layers.GRU(128,return_sequences=True,stateful=True,
                     batch_input_shape=[batchsize,None,maxid],dropout=0.2,recurrent_dropout=0.2),
    keras.layers.GRU(128,return_sequences=True,stateful=True,dropout=0.2,recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(maxid,activation='softmax'))
    
])

In [51]:
class ResetState(keras.callbacks.Callback):
    def on_epoch_begin(self,epoch,logs):
        self.model.reset_states()
        

In [52]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam')

In [54]:
steps_per_epoch = trainsize // batchsize // nstep
model.fit(dataset,steps_per_epoch=steps_per_epoch,epochs=50,callbacks=[ResetState()])

Epoch 1/50


InvalidArgumentError:  Specified a list with shape [32,39] from a tensor with shape [1,39]
	 [[node sequential_4/gru_8/TensorArrayUnstack/TensorListFromTensor (defined at <ipython-input-54-b344a24911ad>:2) ]] [Op:__inference_train_function_17514]

Function call stack:
train_function
